In [ ]:
# ─────────────────────────────────────────────
# 1️⃣ Install Required Libraries
# ─────────────────────────────────────────────
!pip install requests matplotlib opencv-python numpy tqdm python-dotenv huggingface_hub nbconvert --quiet

import os
import base64
import requests
import numpy as np
import matplotlib.pyplot as plt
import cv2
import shutil
from PIL import Image
from io import BytesIO
from tqdm import tqdm
from dotenv import load_dotenv
from google.colab import files
from huggingface_hub import hf_hub_upload
from IPython.display import FileLink, display, HTML

# ─────────────────────────────────────────────
# 2️⃣ Add Notebook Metadata (Optional)
# ─────────────────────────────────────────────
__author__ = "Your Name"
__version__ = "1.0"
print(f"Notebook Author: {__author__}, Version: {__version__}")

# ─────────────────────────────────────────────
# 3️⃣ Load Environment Variables (.env optional)
# ─────────────────────────────────────────────
load_dotenv()
API_URL = os.getenv("API_URL", "http://localhost:8000/predict")
JWT_TOKEN = os.getenv("JWT_TOKEN", "demo-secret")
HEADERS = {
    "Authorization": f"Bearer {JWT_TOKEN}",
    "Content-Type": "application/json"
}

# ─────────────────────────────────────────────
# 4️⃣ Upload Images
# ─────────────────────────────────────────────
uploaded = files.upload()
image_paths = list(uploaded.keys())
print(f"✅ Uploaded {len(image_paths)} image(s)")
os.makedirs("outputs", exist_ok=True)

# ─────────────────────────────────────────────
# 5️⃣ Run Inference and Visualize
# ─────────────────────────────────────────────
for IMG_PATH in tqdm(image_paths, desc="Running Inference"):
    try:
        print(f"\n🔍 Processing: {IMG_PATH}")
        with open(IMG_PATH, "rb") as img_file:
            image_bytes = img_file.read()
        image_base64 = base64.b64encode(image_bytes).decode("utf-8")

        payload = {
            "image_base64": image_base64,
            "normalize": True
        }

        response = requests.post(API_URL, json=payload, headers=HEADERS)

        if response.status_code == 200:
            result = response.json()
            mask = np.array(result["mask"]).reshape((256, 256))
            inference_time = result.get("inference_time", "N/A")
            print(f"⏱ Inference time: {inference_time} seconds")

            mask_binary = (mask > 0.5).astype(np.uint8)
            image_np = np.array(Image.open(IMG_PATH).resize((256, 256)))
            overlay = cv2.addWeighted(image_np, 0.6, cv2.cvtColor(mask_binary * 255, cv2.COLOR_GRAY2RGB), 0.4, 0)

            plt.figure(figsize=(15, 5))
            plt.subplot(1, 3, 1)
            plt.title("Original Image")
            plt.imshow(image_np)
            plt.axis("off")

            plt.subplot(1, 3, 2)
            plt.title("Predicted Mask")
            plt.imshow(mask_binary, cmap='gray')
            plt.axis("off")

            plt.subplot(1, 3, 3)
            plt.title("Overlay")
            plt.imshow(overlay)
            plt.axis("off")

            output_path = f"outputs/pred_{os.path.splitext(IMG_PATH)[0]}.png"
            plt.savefig(output_path)
            plt.show()
            print(f"📸 Saved: {output_path}")
        else:
            print(f"❌ API Error: {response.status_code}")
            print(response.text)

    except Exception as e:
        print(f"❌ Exception occurred: {str(e)}")

# ─────────────────────────────────────────────
# 6️⃣ Zip All Output Images
# ─────────────────────────────────────────────
zip_path = shutil.make_archive("sam_predictions", 'zip', "outputs")
print("📦 All predictions zipped as:", zip_path)
display(FileLink(zip_path))

# ─────────────────────────────────────────────
# 7️⃣ (Optional) Upload to Hugging Face Hub
# ─────────────────────────────────────────────
# from huggingface_hub import login; login()
# hf_hub_upload(
#     folder_path="outputs",
#     repo_id="your-username/sam-results",
#     repo_type="dataset",
#     commit_message="Upload SAM results"
# )

# ─────────────────────────────────────────────
# 8️⃣ (Optional) Export to HTML and PDF
# ─────────────────────────────────────────────
!jupyter nbconvert --to html SAM_Colab_Demo.ipynb --output "SAM_Report.html"
!jupyter nbconvert --to pdf SAM_Colab_Demo.ipynb --output "SAM_Report.pdf"

print("📘 Exported HTML: SAM_Report.html")
print("📘 Exported PDF: SAM_Report.pdf")
